In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
from ta.trend import SMAIndicator
from ta.volatility import BollingerBands
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2010-10-01"
end_date="2021-10-01"
txt = "{:%}"
wind=14
for i in range(0,25):
    try:
        stk=sip[i]
        #df = yf.download(stk, start_date, end_date,interval=intr).dropna()

        dfd = yf.download(stk, start_date, end_date,interval='1d').dropna()
        dfd['Close_1']=dfd['Close'].shift(1)
        dfd['Volume_1']=dfd['Volume'].shift(1)
        
        
        dfd['efid']=ForceIndexIndicator(close=dfd.Close_1,volume=dfd.Volume_1,window=wind,fillna=True).force_index()
        dfd['efi_rsid']=RSIIndicator(close=dfd.efid,window=wind,fillna=True).rsi()
        dfd["kd"]=StochRSIIndicator(close=dfd.Close_1,window=wind,smooth1=3, smooth2=3).stochrsi_k() 
        dfd['bbh']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_hband()
        dfd['bbm']=BollingerBands(close=dfd.Close_1,window=20,window_dev=2,fillna=True).bollinger_mavg()
        dfd['bbg']=dfd['bbh']-dfd.bbm
        
        dfd['sma200']=SMAIndicator(close=dfd.Close_1,window=200,fillna=True).sma_indicator()
        dfd['sma50']=SMAIndicator(close=dfd.Close_1,window=50,fillna=True).sma_indicator()
        dfd['sma20']=SMAIndicator(close=dfd.Close_1,window=20,fillna=True).sma_indicator()
        dfd['sma10']=SMAIndicator(close=dfd.Close_1,window=10,fillna=True).sma_indicator()
        dfd["Chng"]=(dfd.Close-dfd.Open)
        dfd['Chng_1']=dfd.Chng.shift(1)
        #dfd['Chng_smaf']=SMAIndicator(close=dfd.Chng_1,window=5,fillna=True).sma_indicator()
        #dfd['Chng_smas']=SMAIndicator(close=dfd.Chng_1,window=20,fillna=True).sma_indicator()
        #dfd['Vol_smaf']=SMAIndicator(close=dfd.Volume_1,window=5,fillna=True).sma_indicator()
        #dfd['Vol_smas']=SMAIndicator(close=dfd.Volume_1,window=20,fillna=True).sma_indicator()
        #print(dfd.tail)
        dfd=dfd.dropna()
        perd=5

        conditions = [ 
        (dfd['Open']>0)
        &(dfd['Close_1']>dfd.bbm+dfd.bbg)
        #&(dfd['Close_1']<dfd.bbm-dfd.bbg)    
        #&(dfd['kd']<np.percentile(dfd['kd'],100-perd)) 
        &(dfd['kd']>90)
        #(dfd['efi_rsid']<np.percentile(dfd['efi_rsid'],100-perd))  
        #&(dfd['sma10']>dfd['sma200'])
        
            
        #&(dfd['Chng_smaf']>dfd['Chng_smas'])
        #&(dfd['Vol_smaf']>dfd['Vol_smas'])
        #&(dfd['sma20']<dfd['sma50'])
        #&(dfd['sma10']>dfd['sma20'])
        #&(dfd['Open']<dfd['sma10'])
        #(dfd['Open']>dfd['Close_1'])
        #& (dfd['Close_1']>=dfd['psma'])
        ]

        choices = [
        1
        ]
        #print(dfd.head)
        dfd['trade_pass']=np.select(conditions,choices,default=0)
        #print(dfd)
        dfd['trade']=np.select([(dfd['trade_pass']==1),(dfd['trade_pass']==0)],[(-dfd['Chng']),0])
        
        hodl=dfd['Close'][len(dfd.index)-1]-dfd['Open'][0]
        strat=np.sum(dfd.trade)
        base=np.sum(dfd.Chng)
        
        leng=len(dfd.index)
        winr=np.count_nonzero(dfd.trade>0)/np.count_nonzero(dfd.trade_pass==1)
        winrb=np.count_nonzero(dfd.Chng>0)/leng
        trader=np.count_nonzero(dfd.trade_pass==1)/leng
        temp=pd.Series([stk,hodl,base,strat,winrb,winr,trader])
        dfout=dfout.append(temp,ignore_index=True)
        #print(df.tail())
    except:
        pass
dfout.columns=['stk','hold','base_return','strat_return','base_winrate','strat_winrate','trader']
print(dfout)
print(txt.format(np.percentile(dfout.base_winrate,50)))
print(txt.format(np.percentile(dfout.strat_winrate,50)))
print(txt.format(np.percentile(dfout.strat_winrate,50)-(np.percentile(dfout.base_winrate,50))))
#print(txt.format(np.average(dfout.strat_traderate)))
#dfout.to_csv('stochrsi_week_edge.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements